# Extractive Summarization

#### Required Libraries

In [1]:
import os
import sys
import json
import pickle
from tqdm import tqdm
from nltk.corpus import stopwords
import nltk
import operator
import numpy as np
import pandas as pd
import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

In [4]:
# Intializing the Word2Vec Model, download the file from https://nlp.stanford.edu/projects/glove/
# Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, uncased, 50d, 100d, 200d, & 300d vectors, 822 MB download): glove.6B.zip 
# Unzip the file then run: python3 -m gensim.scripts.glove2word2vec --input  glove.6B.300d.txt --output glove.6B.300d.w2vformat.txt
model = gensim.models.KeyedVectors.load_word2vec_format('data/glove.6B.300d.w2vformat.txt')

In [5]:
word_freq_map = {}
with open("data/arxiv-release/vocab", 'r') as vocab_file:
    lines = vocab_file.readlines()
    for line in lines:
        word_freq_map[line.split()[0]] = int(line.split()[1])
    
stop_list = sorted(word_freq_map.items(), key=operator.itemgetter(1), reverse=True)[:150]
cache = {}

#### Read the full training data and split it into smaller chunks

In [6]:
def read_data_and_split(train_corpus_path):
    train_data_map = {}
    file_no = 0
    with open(train_corpus_path, 'r') as train_data_file:
        line_count = 0
        while file_no < 11:
            if line_count < 20000:
                line_data = train_data_file.readline()
                if line_data:
                    line_map = json.loads(line_data)
                    article_id = line_map['article_id']
                    del line_map['article_id']
                    train_data_map[article_id] = line_map
                    line_count += 1
                else:
                    break
            else:
                with open(train_corpus_path.rsplit('/', 1)[0] + "/SplitTrain/" + "train_" + str(file_no) + ".pickle", 'wb') as train_file:
                    pickle.dump(train_data_map, train_file, protocol=pickle.HIGHEST_PROTOCOL)
                print("File ", file_no, " Done")
                train_data_map.clear()
                file_no += 1
                line_count = 0
        with open(train_corpus_path.rsplit('/', 1)[0] + "/SplitTrain/" + "train_" + str(file_no) + ".pickle", 'wb') as train_file:
            pickle.dump(train_data_map, train_file, protocol=pickle.HIGHEST_PROTOCOL)

#### Loads a particular pickle file of the training data into memory

In [7]:
def load_data_from_pickle(train_corpus_path):
    data_path = train_corpus_path.rsplit('/', 1)[0] + "/SplitTrain"
    data_map = {}
    with open(data_path + "/" + "train_0.pickle", 'rb') as handle:
        data_map = pickle.load(handle)
    return data_map

#### Gets all the sentences of the article along with its metadata

In [75]:
def get_sentences_with_metadata(data_map):
    full_text = []
    sentence_metadata = []
    list_of_sentences = []
    summary_list = []
    abstract_list = []
    c = 0
    file_number = 1
    for article_id, data in tqdm(data_map.items()):
        if c < 100:
            abstract_list.append(data['abstract_text'])
            section_data = data['sections']
            section_names = data['section_names']
            for i, section in enumerate(section_data):
                for line in section:
                    split_line = line.split('.')
                    for l in split_line:
                        list_of_sentences.append(l)
                        sentence_metadata.append(section_names[i])
            summary_list.append(do_stuff_and_get_summary(list_of_sentences))
        else:
            c = 0
            write_summary_and_abstract_to_file(summary_list, abstract_list, file_number)
            print("File ", file_number, " done")
            file_number += 1
            summary_list.clear()
            abstract_list.clear()
        c += 1
        
    return list_of_sentences, sentence_metadata

#### The Following 2 functions are used for Preprocessing of a given text

In [9]:
def is_ascii(word):
    """
    Checks if word is ascii or not
    :param word: token
    :return: Boolean
    """
    valid = True
    try:
        word = word.encode('ascii')
    except UnicodeEncodeError:
        valid = False
    return valid

In [10]:
def get_processed_tokens(sentence):
    punc_map = {}
    punc_map = punc_map.fromkeys('!"\'()*+,;<>[]^`{|}~:=%&_#?-$/', ' ')
    table = str.maketrans(punc_map)
    tokens = sentence.lower().translate(table).split()
    stop_words = set(stopwords.words('english')) 
    stop_words = list(stop_words) + stop_list
    cleaned_tokens = [word for word in tokens if word not in stop_words and is_ascii(word) and '@' not in word and '\\' not in word and len(word) > 1]            
    return cleaned_tokens

#### Gets the processed sentences for each sentence of the article

In [11]:
def make_processed_sentences(list_of_sentences):
    processed_sentences = []
    for sentence in list_of_sentences:
        if isinstance(sentence, list):
            sentence = " ".join(sentence)
        processed_sentences.append(get_processed_tokens(sentence))
    return processed_sentences

#### Gives the number of words common between given 2 sentences

In [12]:
def get_no_of_common_word(sentence1, sentence2):
    common_count = 0
    for s1 in sentence1:
        for s2 in sentence2:
            if s1 == s2:
                common_count += 1
    return common_count

In [13]:
def get_word_vec_sim(sentence1, sentence2):
    score = 0
    for word1 in sentence1:
        for word2 in sentence2:
            try:
                temp = cache[word1+word2]
            except:
                try:
                    temp = model.similarity(word1, word2)
                    cache[word1+word2] = temp
                    cache[word2+word1] = temp
                except:
                    cache[word1+word2] = 0
                    cache[word2+word1] = 0
                    temp = 0
            score += temp
    return score

#### Generic scoring function which gives a score between 2 sentences

In [14]:
def scoring(sentence1, sentence2, metadata):
    len_normalize = len(sentence1) + len(sentence2) + 1 # Normalizing by length of vector
    common_words = get_no_of_common_word(sentence1, sentence2)
    word_vec_score = get_word_vec_sim(sentence1, sentence2)
    score = common_words / 2*len_normalize + word_vec_score / len_normalize
    return score

#### Makes the graph which has relations between every pair of sentences

In [15]:
def make_graph(processed_sentences, metadata):
    sentence_graph = np.zeros(shape=(len(processed_sentences), len(processed_sentences)))
    sentence_common_graph = np.zeros(shape=(len(processed_sentences), len(processed_sentences)))
    
    for i in range(len(processed_sentences)):
        for j in range(len(processed_sentences)):
            sentence1 = processed_sentences[i]
            sentence2 = processed_sentences[j]
            if i == j:
                sentence_graph[i][j] = 0
                sentence_common_graph[i][j] = 0
            else:
                sentence_graph[i][j] = scoring(sentence1, sentence2, metadata)
                sentence_common_graph[i][j] = get_no_of_common_word(sentence1, sentence2)
    return sentence_graph, sentence_common_graph

#### Following functions are different ways to give a score to a sentence

##### (1) Aggregation

In [16]:
def calculate_scores(sentence_graph):
    scores = np.zeros(len(sentence_graph))
    for i,sentence in enumerate(sentence_graph):
        scores[i] = sum(sentence_graph[i])
    return scores

##### (2) Page Rank

In [17]:
def calculate_pagerank_scores(sentence_graph):
    N = len(sentence_graph)
    d = 0.15   # PageRank Hyperparameter
    pagerank_scores = np.ones(N)
    
    out_degree = np.zeros(N)
    for i in range(N):
        for j in range(N):
            if sentence_graph[i][j]:
                out_degree[i] += sentence_graph[i][j]
    
    for i in range(N):
        score = 0
        for j in range(N):
            if sentence_graph[j][i]:
                score += (pagerank_scores[j] / out_degree[j])
        pagerank_scores[i] = (d / N) + (1 - d) * score
    return pagerank_scores    

#### Ranks the sentences based on any one of the above scoring methods and return the Summary

In [18]:
def rank_sentences_and_make_summary2(sentences, processed_sentences, sentence_graph, scores):
    scores_indices = np.argsort(scores)
    ordered_sentences = scores_indices[::-1]
    summary = []
    for i in range(5):
        summary.append(sentences[ordered_sentences[i]])
#         print(ordered_sentences[i], scores[ordered_sentences[i]])
#         print(processed_sentences[ordered_sentences[i]])
    return summary

In [19]:
def rank_sentences_and_make_summary(sentences, processed_sentences, sentence_graph, scores, summary_length):
    summary = []
    for i in range(5): # Number of Sentences we want in the summary
        score_indices = np.argsort(scores)
        selected_index = score_indices[-1]
        summary.append(sentences[selected_index]) # Adding highest score sentence to summary
        mean_score = np.mean(sentence_graph)
        to_decrease = []
        # Calculated mean similarity score. If selected sentence and another sentence have
        # high similarity, the score of the second sentence should be reduced.
        # Here, have chosen to use 1.5 * mean_score as the threshold, and divided score in half.
        for iterator in range(len(processed_sentences)):
            if sentence_graph[iterator][selected_index] > 1.5 * mean_score:
                to_decrease.append(iterator)
            if sentence_graph[selected_index][iterator] > 1.5 * mean_score:
                to_decrease.append(iterator)
        for sentence in set(to_decrease):
            # Should be changed based on the number of sentences needed in the summary
            scores[sentence] /= (1 + 1.0 / summary_length) # Reduced score by half, to on average prevent from being picked.
        scores[selected_index] = 0
    return summary
        

#### Main Program which calls the above defined functions

In [21]:
# train_corpus_path = "data/arxiv-release/train.txt"
train_corpus_path = "/media/kaushik/Studies/IIITH/3_ThirdSem/IRE/Major Project/arxiv-release/arxiv-release/train.txt"
# read_data_and_split(train_corpus_path)

In [36]:
data_map = load_data_from_pickle(train_corpus_path)

dict_keys(['1405.3379', '0901.1147', 'nlin0608019', '0903.5449', 'hep-ph0605279', '1405.7337', '1607.05158', '1511.00051', '1002.1288', '0706.1962', '1008.0881', 'hep-ph0605134', 'astro-ph9608054', 'cond-mat0506313', 'astro-ph0202198', '0907.2161', 'cond-mat9907309', '0909.2256', '1508.06930', '1007.1918', '1208.5351', 'astro-ph0310704', '1501.01494', '1606.05955', 'cs0003079', '0905.2691', '0903.4724', '1211.7095', '1104.4750', '0908.1913', '1108.0797', 'hep-ph0102030', '1305.7010', 'quant-ph0605207', '1105.2710', 'astro-ph0511483', 'nlin0005053', 'physics0404057', '1604.08362', '1402.0371', '1006.1803', '1602.05000', '0810.4598', '1608.04532', 'math-ph0407034', '0904.1879', '0909.4715', 'hep-ph0405167', 'astro-ph9506066', '0711.1717', '1205.5710', '1501.02852', 'gr-qc0006038', 'math9903022', '1302.3277', '1703.08158', '1701.00774', '1205.0614', '1510.00115', '1210.3325', 'physics0109012', 'astro-ph9905371', '1006.3484', '1609.00244', 'nucl-th9903007', 'hep-ph0409031', '0707.4010', '1

In [38]:
len(data_map)

20000

In [76]:
def do_stuff_and_get_summary(list_of_sentences):
    list_of_sentences = [sentence.strip() for sentence in list_of_sentences if len(sentence) > 1]
    processed_sentences = make_processed_sentences(list_of_sentences)
    sentence_graph, sentence_common_graph = make_graph(processed_sentences, sentence_metadata)
    sentence_scores = calculate_scores(sentence_graph)
    sentence_page_scores = calculate_pagerank_scores(sentence_common_graph)
    sentence_score_final = [sentence_scores[i] * (sentence_page_scores[i]+1)for i in range(len(sentence_scores))]
    summary_length = 10
    summary = rank_sentences_and_make_summary(list_of_sentences, processed_sentences, sentence_graph, sentence_score_final, summary_length)
    return summary

In [77]:
def write_summary_and_abstract_to_file(summary_list, abstract_list, file_number):
    map_of_abstract_summary = {}
    
    abstract_map = {}
    for i, abstract in enumerate(abstract_list):
        abstract_map[i] = abstract
    
    summary_map = {}
    for i, summary in enumerate(summary_list):
        summary_map[i] = summary
    
    with open("data/map/abstract_file_" + str(file_number) + ".pickle", 'wb') as abs_file:
        pickle.dump(abstract_map, abs_file, protocol=pickle.HIGHEST_PROTOCOL)
    with open("data/map/summary_file_" + str(file_number) + ".pickle", 'wb') as sum_file:
        pickle.dump(summary_map, sum_file, protocol=pickle.HIGHEST_PROTOCOL)

#### Call this function and this function is now changed to generate summaries for all articles and store them to files

In [ ]:
list_of_sentences, sentence_metadata = get_sentences_with_metadata(data_map)




  0%|          | 0/20000 [00:00<?, ?it/s]


  0%|          | 1/20000 [00:01<9:18:40,  1.68s/it]


  0%|          | 2/20000 [00:07<15:35:54,  2.81s/it]


  0%|          | 3/20000 [00:24<39:15:01,  7.07s/it]


  0%|          | 4/20000 [00:52<74:55:47, 13.49s/it]/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):



  0%|          | 5/20000 [01:59<163:48:35, 29.49s/it]


  0%|          | 6/20000 [03:47<294:47:11, 53.08s/it]


  0%|          | 7/20000 [06:39<492:53:12, 88.75s/it]


  0%|          | 8/20000 [10:06<690:18:08, 124.30s/it]


  0%|          | 9/20000 [13:27<817:33:35, 147.23s/it]


  0%|          | 10/20000 [19:02<1130:29:26, 203.59s/it]


  0%|          | 11/20000 [27:07<1598:36:59, 287.91s/it]


  0%|          | 12/20000 [34:42<1876:28:44, 337.97s/i

In [35]:
list_of_sentences, sentence_metadata = get_sentences_with_metadata(data_map)
list_of_sentences = [sentence.strip() for sentence in list_of_sentences if len(sentence) > 1]

In [33]:
processed_sentences = make_processed_sentences(list_of_sentences)

In [25]:
sentence_graph, sentence_common_graph = make_graph(processed_sentences, sentence_metadata)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [26]:
sentence_scores = calculate_scores(sentence_graph)

In [27]:
sentence_page_scores = calculate_pagerank_scores(sentence_common_graph)

In [28]:
sentence_score_final = [sentence_scores[i] * (sentence_page_scores[i]+1)  for i in range(len(sentence_scores))]

In [29]:
summary_length = 10
summary = rank_sentences_and_make_summary(list_of_sentences, processed_sentences, sentence_graph, sentence_scores, summary_length)
summary

['the hamiltonian modeling this system consists of the ground and one excited state of the molecule and a quasi - continuum describing the conduction band together with one vibrational coordinate @xmath15 here @xmath16 can be equal to @xmath17 for the ground state , @xmath18 for the excited state , and @xmath19 for the quasi - continuum',
 'first , the vibrational populations in the excited state of the molecule no longer only decay into the quasi - continuum states but also relax within the excited state ( see fig',
 'therefore the total hamiltonian consists of three terms  the system part @xmath1 , the bath part @xmath2 , and the system - bath interaction @xmath3 : @xmath4 the rdm @xmath5 is obtained from the density matrix of the full system by tracing out the degrees of freedom of the environment',
 'the electronic probabilities in the quasi - continuum are given as @xmath30 where @xmath31 is the initial vibronic distribution in the excited state and @xmath32 and @xmath33 are the v

In [30]:
summary = rank_sentences_and_make_summary(list_of_sentences, processed_sentences, sentence_graph, sentence_score_final, summary_length)
summary

['the hamiltonian modeling this system consists of the ground and one excited state of the molecule and a quasi - continuum describing the conduction band together with one vibrational coordinate @xmath15 here @xmath16 can be equal to @xmath17 for the ground state , @xmath18 for the excited state , and @xmath19 for the quasi - continuum',
 'therefore the total hamiltonian consists of three terms  the system part @xmath1 , the bath part @xmath2 , and the system - bath interaction @xmath3 : @xmath4 the rdm @xmath5 is obtained from the density matrix of the full system by tracing out the degrees of freedom of the environment',
 'first , the vibrational populations in the excited state of the molecule no longer only decay into the quasi - continuum states but also relax within the excited state ( see fig',
 'recently @xcite the electron injection from a chromophore to a semiconductor conduction band was described using the time - dependent schrdinger equation , thus neglecting relaxation p

In [31]:
summary = rank_sentences_and_make_summary(list_of_sentences, processed_sentences, sentence_graph, sentence_page_scores, summary_length)
summary
# make_processed_sentences(summary)

['if one assumes bilinear system - bath coupling with system part @xmath8 and bath part @xmath9 @xmath10 one can take advantage of the following decomposition @xcite : @xmath11   +    [ \\lambda\\rho , k]+   [ k,\\rho\\lambda^{\\dagger } ]',
 'recently @xcite the electron injection from a chromophore to a semiconductor conduction band was described using the time - dependent schrdinger equation , thus neglecting relaxation processes',
 'we are aware that this is only a minimal model but hope that it catches the effects of dissipation on the electron injection process',
 'the hamiltonian modeling this system consists of the ground and one excited state of the molecule and a quasi - continuum describing the conduction band together with one vibrational coordinate @xmath15 here @xmath16 can be equal to @xmath17 for the ground state , @xmath18 for the excited state , and @xmath19 for the quasi - continuum',
 'to be able to study the effects of dissipation']